## CNN Architecture

In [45]:
import torch

from torch import nn, optim
from torch.utils.data import Dataset
from torch.nn import functional as F
from torchvision.datasets import ImageFolder
from torchvision import transforms, models

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# DEVICE = torch.device('cuda' if torch.cuda.is_available else 'cpu')
DEVICE = 'cpu'

In [46]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        # 224, 224
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
    
    def forward(self, x):
        out = self.pool1(F.relu(self.conv1(x)))
        out = self.pool2(F.relu(self.conv2(out)))
        out = self.pool3(F.relu(self.conv3(out)))
        
        out = self.flatten(out)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        
        return torch.sigmoid(self.fc3(out))

## Dataset class

In [47]:
class DataSet(Dataset):
    def __init__(self, path, transformations=None):
        self.train_data = ImageFolder(f'{path}/train', transform=transformations)
        self.test_data = ImageFolder(f'{path}/test', transform=transformations)
    
    def __len__(self):
        return len(self.train_data) + len(self.test_data)
    
    def __getitem__(self, index):
        # return the features and target separated
        return self.dataset[index, :-1], self.dataset[index, -1]

## DataLoader

In [48]:
class DataLoader():
    def __init__(self, train, test, batch_size, shuffle=True):
        self.train_data = train
        self.test_data = test
        self.batch_size = batch_size
        self.shuffle = shuffle
    
    def load(self):
        loaders = (torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size, shuffle=self.shuffle),
                   torch.utils.data.DataLoader(self.test_data, batch_size=self.batch_size, shuffle=self.shuffle))
        
        return loaders

In [49]:
def training_loop(model, train, test, n_epochs, optimizer, loss_fn, print_plot=True):
    train_accuracy = torch.zeros(n_epochs)
    test_accuracy = torch.zeros(n_epochs)

    for epoch in range(n_epochs):
        for example, label in train:
            example = example.to(DEVICE)
            label = label.to(DEVICE)

            # return a list instead of a list of one element lists
            label = label.unsqueeze(1)

            # predict and compute loss
            output = model(example.float())
            loss = loss_fn(output, label.float())

            optimizer.zero_grad() # zero gradient (make errors to 0)
            loss.backward()
            optimizer.step() # change weights

        with torch.no_grad():
            for loader, accuracy in [(train, train_accuracy), (test, test_accuracy)]:
                correct = 0
                total = 0

                for examples, labels in loader:
                    examples = examples.to(device=DEVICE)
                    labels = labels.to(device=DEVICE).view(-1, 1)

                    outputs = model(examples.float())
                    predicted = torch.round(outputs)

                    total += labels.shape[0]
                    correct += (predicted == labels).sum()

                accuracy[epoch] = correct / total
        
        # print the accuracy of every tenth epoch
        if (epoch+1) % 10 ==0:
            print(f'Epoch {epoch+1})', 
                  f'Train Accuracy: {train_accuracy[epoch]}',
                  f'Test Accuracy: {test_accuracy[epoch]}') 
        
    if print_plot:
        epochs = range(n_epochs)

        #Ploting both curves, train and val 
        plt.plot(epochs, train_accuracy, 'g', label='Training accuracy')
        plt.plot(epochs, test_accuracy, 'b', label='Test accuracy')
        plt.title('Training and Test loss')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

## Training

In [50]:
transformations = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

data = DataSet('./task_4_data', transformations)
train_loader, test_loader = DataLoader(data.train_data, data.test_data, batch_size=32).load()

In [53]:
model = CNN().to(DEVICE)
n_epochs = 24
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

training_loop(
    model,
    train_loader,
    test_loader,
    n_epochs,
    optimizer,
    loss_fn
)

KeyboardInterrupt: 